In [7]:
from glob import glob
from pathlib import Path
import cv2
import albumentations as A
from fl_tissue_model_tools.transforms import get_elastic_dual_transform
from fl_tissue_model_tools.preprocessing import get_augmentor, augment_images

In [8]:
'''
This notebook is used to test the augmentation pipeline for binary segmentation.
https://ai/docs/getting_started/image_augmentation/
Compose:
- elastic transform from fl tools transforms module (p=0.9)
- one of (probability=0.8): (not on mask)
    - randomgamma (probability=0.25)
    - randombrightnesscontrast (probability=0.25)
    - randomtonecurve (probability=0.25)
    - clahe (probability=0.25)
- one of (probability=0.75):
    - verticalflip (probability=0.5)
    - randomrotate90 (probability=0.5)
'''


albumentations_transform = A.Compose([
    A.Rotate(p=0.5, border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0),
    A.RandomCrop(height=572, width=572),
    A.Flip(p=0.5),
    A.OneOf([
            A.MultiplicativeNoise(p=0.5),
            A.AdvancedBlur(p=0.5)
    ], p=0.8),
    A.OneOf([
        A.RandomGamma(p=0.2),
        A.RandomBrightnessContrast(p=0.4),
        A.RandomToneCurve(scale=0.75, p=0.2),
        A.CLAHE(p=0.2)
    ], p=0.75)
])

elastic_transform = get_elastic_dual_transform(
    grid_width_range=[4,8],
    grid_height_range=[4,8],
    magnitude_range=[7, 9]
)

transform_list = (albumentations_transform, elastic_transform)
augmentor = get_augmentor(transform_list)

In [9]:
print(type(transform_list))

<class 'tuple'>


In [10]:
# Change this to match your data paths
images = [cv2.imread(f, 0) for f in glob('set_1/images/*.tif')]
masks = [cv2.imread(f, 0) for f in glob('set_1/masks/*.tif')]

In [11]:
Path('transforms_output').mkdir(parents=True, exist_ok=True)

In [12]:
augmented = augment_images(images, masks, augmentor)
n = 1
for img, mask in augmented:
	cv2.imwrite(f"img_{n}", img)
	cv2.imwrite(f"mask{n}", mask)
	n += 1

<class 'function'>


TypeError: 'function' object is not iterable